# Пример цепочки сравнения ответов с помощью structured output
COT цепочка, которую мы использовали в [3_evaluation.ipynb](3_evaluation.ipynb), возможно
скоро устареет. Здесь приведен пример построения цепочки на LCEL, который более свежий.

In [ ]:
!pip install gigachain-community

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate


class Grade(BaseModel):
    """Оценка теста"""
    explanation: str = Field(description="пошаговое рассуждение здесь")
    grade: int = Field(description="Если ответ не правильный - 0, если правильный - 1")


prompt = PromptTemplate.from_template(
    """Ты учитель, оценивающий тест.

Тебе дан вопрос, корректный ответ и ответ студента. Тебе нужно оценить ответ студента как ПРАВИЛЬНЫЙ или НЕПРАВИЛЬНЫЙ, основываясь на корректном ответе.
Опиши пошагово своё рассуждение, чтобы убедиться, что твой вывод правильный. Избегай просто указывать правильный ответ с самого начала.

Вот базовая информация из конкретной области этого теста:
GigaChat - это большая языковая модель (LLM) от Сбера.
GigaChat API (апи) - это API для взаимодействия с GigaChat по HTTP с помощью REST запросов.
GigaChain - это SDK на Python для работы с GigaChat API. Русскоязычный форк библиотеки LangChain.
GigaGraph - это дополнение для GigaChain, который позволяет создавать мультиагентные системы, описывая их в виде графов.
Обучение GigaChat выполняется командой разработчиков. Дообучение и файнтюнинг для конечных пользователей на данный момент не доступно.
Для получения доступа к API нужно зарегистрироваться на developers.sber.ru и получить авторизационные данные.

Опирайся на эту базовую информацию, если тебе не хватает информации для проверки теста.

Пример формата:
QUESTION: здесь вопрос
TRUE ANSWER: здесь корректный ответ
STUDENT ANSWER: здесь ответ студента

Тебе будем дан только один ответ студента, не несколько.
Оценивай ответ студента ТОЛЬКО на основе их фактической точности.
Игнорируй различия в пунктуации и формулировках между ответом студента и правильным ответом.
Ответ студента может содержать больше информации, чем правильный ответ, если в нём нет противоречивых утверждений, то он корректен.
Оценивай ответ с помощью Grade.
Начнём!

QUESTION: "{query}"
TRUE ANSWER: "{context}"
STUDENT ANSWER: "{result}" """  # noqa
)

In [3]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

ch = prompt | llm.with_structured_output(Grade)

Сравнение неправильного ответа

In [4]:
ch.invoke({"query": "Кто главный герой книги", "context": "Кот", "result": "Собака"})

Grade(explanation='1. Вопрос спрашивает о главном герое книги.\\n2. Правильный ответ на вопрос - "Кот".\\n3. Ответ студента - "Собака".\\n4. Ответ студента не совпадает с правильным ответом.\\n5. Следовательно, ответ студента является неправильным.', grade=0)

Сравнение правильного ответа

In [5]:
ch.invoke({"query": "Кто главный герой книги", "context": "Кот", "result": "Котик"})

Grade(explanation="1. Вопрос спрашивает о главном герое книги.\\n2. Корректный ответ - 'Кот'.\\n3. Ответ студента - 'Котик'.\\n4. Слово 'Котик' является уменьшительно-ласкательной формой слова 'Кот'.\\n5. Смысл ответа студента совпадает с корректным ответом, так как 'Котик' и 'Кот' обозначают одно и то же животное.\\n6. Ответ студента не содержит противоречивой информации.\\n7. Следовательно, ответ студента можно считать правильным.", grade=1)